# Finding the MRI brain tumor detection dataset

Let's find the dataset in this link: https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection

# Import packages

In [28]:
#!pip install scikit-learn opencv-python 

In [29]:
#pip install torch==2.0+cu117 --user -f https://download.pytorch.org/whl/cu117/torch_stable.html

In [30]:
#!pip install torchvision==0.15.0

In [31]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
#import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import sys
from torchvision import models,transforms,datasets
import time
import os
import torch.nn.functional as F
from torchvision.datasets.folder import default_loader
import torchvision
from sklearn .metrics import roc_auc_score

In [32]:
print(torch.cuda.is_available()) # this should return true
torch.cuda.get_device_name() # this should return your graphics card name. Ex) 'NVIDIA RTX A4000'

True


'NVIDIA RTX A4000'

In [33]:
# # importing the zipfile module 
"""
from zipfile import ZipFile 

 # loading the temp.zip and creating a zip object 
with ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis_ddsm.zip", 'r') as zObject: 
    zObject.extractall() 
"""

'\nfrom zipfile import ZipFile \n\n # loading the temp.zip and creating a zip object \nwith ZipFile("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis_ddsm.zip", \'r\') as zObject: \n    zObject.extractall() \n'

In [34]:
import pandas as pd
dicom_data = pd.read_csv("C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\dicom_info.csv")

full_mammogram_images = dicom_data[dicom_data.SeriesDescription == 'full mammogram images'].image_path
full_mammogram_images.head()

1     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.248386...
2     CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.267213...
11    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.210396...
12    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.749566...
15    CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.987658...
Name: image_path, dtype: object

In [35]:
# load the mass dataset
mass_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_train_set.csv')
mass_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\mass_case_description_test_set.csv')

#mass_train.head()
mass_train.iloc[0][11].split("/")[2]

# fix image paths
def fix_image_path(data):
 #   """correct dicom paths to correct image paths"""
  for index, img in enumerate(data.values):
    img_name = img[11].split("/")[2]
    new_path = "C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\jpeg\\" + img_name
    data.iloc[index,11] = new_path
      
# apply to datasets
fix_image_path(mass_train)
fix_image_path(mass_test)

C:\Users\ae423\AppData\Local\Temp\ipykernel_21928\1692899711.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mass_train.iloc[0][11].split("/")[2]


In [36]:
calc_train = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_train_set.csv')
calc_test = pd.read_csv('C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\csv\\calc_case_description_test_set.csv')

fix_image_path(calc_train)

In [37]:
temp_calc_train = calc_train[["image file path","pathology"]]
calc_normal = []
calc_malignant = []
for row,col in temp_calc_train.iterrows():
    if col["pathology"] == "MALIGNANT":
        calc_malignant.append(col["image file path"])
    else:
        calc_normal.append(col["image file path"])

print(len(calc_normal))
print(len(calc_malignant))
print(len(temp_calc_train))

1002


544
1546


In [38]:
new_df = mass_train[["image file path","pathology"]]
new_df

normal_path = []
malignant_path = []
for row,col in new_df.iterrows():
    if col["pathology"] == "MALIGNANT":
        # put this file to the malignant folder
        malignant_path.append(col["image file path"])
    else:
        normal_path.append(col["image file path"])

print(len(normal_path))
print(len(malignant_path))
print(len(new_df))

681
637
1318


In [39]:
from sklearn.model_selection import train_test_split

last_df = pd.concat([temp_calc_train,new_df])
temp,test = train_test_split(last_df, test_size=0.1,shuffle= True)
train,val = train_test_split(temp, test_size=0.1,shuffle= True)

print(len(train))
print(len(val))
print(len(test))

2319
258
287


In [40]:

import shutil

def write_into_train_folder():
    for row,col in train.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train\\normal")
                # put it into normal folder


def write_into_val_folder():
    for row,col in val.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val\\normal")
                # put it into normal folder


def write_into_test_folder():
    for row,col in test.iterrows():
        for img_name in os.listdir(col["image file path"]):
            img_path = col["image file path"] + "\\" + img_name
            if col["pathology"] == "MALIGNANT":
                # put this file to the malignant folder
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\malignant")
            else:
                shutil.copy(img_path,"C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test\\normal")
                # put it into normal folder

write_into_train_folder()
write_into_val_folder()
write_into_test_folder()


In [ ]:
"""
import os 
#os.makedirs("cbis-train/normal")
os.makedirs("cbis-train/malignant")
os.makedirs("cbis-val/normal")
os.makedirs("cbis-val/malignant")
os.makedirs("cbis-test/normal")
os.makedirs("cbis-test/malignant")

def write_inside_folder(df):
    for folder in df:
        for file in os.listdir(folder):
            cv2.imwrite(folder,file)

#write_inside_folder(train)
#write_inside_folder(val)
#write_inside_folder(test)
"""

'\nimport os \n#os.makedirs("cbis-train/normal")\nos.makedirs("cbis-train/malignant")\nos.makedirs("cbis-val/normal")\nos.makedirs("cbis-val/malignant")\nos.makedirs("cbis-test/normal")\nos.makedirs("cbis-test/malignant")\n\ndef write_inside_folder(df):\n    for folder in df:\n        for file in os.listdir(folder):\n            cv2.imwrite(folder,file)\n\n#write_inside_folder(train)\n#write_inside_folder(val)\n#write_inside_folder(test)\n'

In [ ]:
import wandb
run = wandb.init(
    # Set the project where this run will be logged
    project="tezim",
    # Track hyperparameters and run metadata
)
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: alphanhilal (alphanhilal-university-of-sussex). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
class BCDataset(datasets.ImageFolder):
  def __init__(self, root, loader=default_loader, is_valid_file=None):
    super(BCDataset, self).__init__(root=root, loader=loader, is_valid_file=is_valid_file)

  def __getitem__(self, index):
    
    
    im = cv2.imread(image_path)
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    new_img = clahe.apply(imgray)
    sample = torchvision.transforms.ToTensor()(new_img)
    return sample, target

In [ ]:
def build_dataset(batch_size):
   
    transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
    
    train_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train'
    #data = BCDataset(train_dir,transforms_train)
    train_dataset = datasets.ImageFolder(train_dir, transforms_train)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    test_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test'
    #data_test = BCDataset(test_dir,transforms_train)
    test_dataset = datasets.ImageFolder(test_dir, transforms_train)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    val_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val'
    #data_val = BCDataset(val_dir,transforms_train)
    val_dataset = datasets.ImageFolder(val_dir, transforms_train)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_dataloader,test_dataloader,val_dataloader


def build_resnet():
    model_ft = models.resnet50(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs, 2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

def build_densenet():
    model_ft = models.densenet169(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs, 2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)

def build_vggnet():
    model_ft = models.vgg19(weights='IMAGENET1K_V1')
    for param in model_ft.parameters():
        param.requires_grad = False
    num_ftrs = model_ft.fc.in_features

    model_ft.fc = nn.Linear(num_ftrs, 2)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model_ft.to(device)


def build_inception():
    model = models.inception_v3(pretrained=True)
    model.aux_logits = False

    for parameter in model.parameters():
        parameter.requires_grad = False

    model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 10),
    nn.Linear(10, 2)
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    return model.to(device)


def inception_dataset(batch_size):
    train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # ToTensor : [0, 255] -> [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
    
    train_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-train'
    train_dataset = datasets.ImageFolder(train_dir, train_transform)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    test_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-test'
    test_dataset = datasets.ImageFolder(test_dir, train_transform)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    val_dir = 'C:\\Users\\ae423\\OneDrive - University of Sussex\\Desktop\\AlphanElmasDissertation\\cbis-val'
    val_dataset = datasets.ImageFolder(val_dir, train_transform)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_dataloader,test_dataloader,val_dataloader
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = nn.CrossEntropyLoss()(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader),



def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        train_dataloader,test_dataloader,val_dataloader = build_dataset(config.batch_size)
        network = build_resnet()
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, train_dataloader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})           


def test(model):
    model.eval()
    y_true = []
    y_pred = []

    train_dataloader,test_dataloader,val_dataloader = build_dataset(32)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for images, labels in test_dataloader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_pred.extend(predicted.cpu().numpy())
        y_true.extend(labels.cpu().numpy())
        

    # Convert lists to tensors for calculation
    y_true_tensor = torch.tensor(y_true)
    y_pred_tensor = torch.tensor(y_pred)

    # Calculating precision, recall, and F1 score using PyTorch
    TP = ((y_pred_tensor == 1) & (y_true_tensor == 1)).sum().item()
    FP = ((y_pred_tensor == 1) & (y_true_tensor == 0)).sum().item()
    FN = ((y_pred_tensor == 0) & (y_true_tensor == 1)).sum().item()

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    auc = np.round(roc_auc_score(y_true, y_pred), 3)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    
    wandb.log({"precision":precision})
    wandb.log({"recall":recall})
    wandb.log({"f1 score":f1 score})
    wandb.log({"AUC":auc})

In [ ]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'learning_rate': {
        "values" : [0.001, 0.0001]
      },
      'batch_size': {
        "values" : [32,64,128,256]
      },
      "epochs" : {
        "values" : [10,20]
      }
    }

sweep_config['parameters'] = parameters_dict


import pprint
pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="tezim")

wandb.agent(sweep_id, train, count=5)


{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [32, 64, 128, 256]},
                'epochs': {'values': [10, 20]},
                'learning_rate': {'values': [0.001, 0.0001]},
                'optimizer': {'values': ['adam', 'sgd']}}}
Create sweep with ID: icgpo079
Sweep URL: https://wandb.ai/alphanhilal-university-of-sussex/tezim/sweeps/icgpo079


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: mtsld2ra with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	optimizer: adam


Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\site-packages\wandb\sdk\wandb_run.py", line 303, in check_stop_status
    self._loop_check_status(
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\site-packages\wandb\sdk\wandb_run.py", line 233, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\site-packages\wandb\sdk\interface\interface.py", line 907, in deliver_stop_status
    return self._deliver_stop_status(status)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\envs\AlphanElmas\Lib\site-packages\wandb\sdk\interface\interface_shared.py", line 487, in _

batch loss,▇█▅▃▄▃▂▂▂▂▂▂▂▂▂▄▂▃▂▃▂▁▂▂▃▂▂▂▁▂▁▂▃▃▂▁▂▁▂▁
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▂▂▂▂▁▂▁▁
batch loss,0.61021
epoch,9
loss,0.67455


wandb: Agent Starting Run: eovdv659 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.001
wandb: 	optimizer: sgd


batch loss,▆▇▅▅▄▆█▅▆▅▆▆▄▇▅▂▇█▃▄▄▄▄▄▅▄▂▁▃▇▆▃▅▂▅▃▆▆▆▆
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇█▅▅▃▄▄▁▆
batch loss,0.7062
epoch,9
loss,0.69234


wandb: Agent Starting Run: xaguwnak with config:
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: adam


In [ ]:
#model_ft = train_model(model_ft, criterion, optimizer_ft,num_epochs=3)

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x00000236D8BDE710>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 236ddeab990, raw_cell="#model_ft = train_model(model_ft, criterion, optim.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X42sZmlsZQ%3D%3D>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x00000236D8BDE710>> (for post_run_cell), with arguments args (<ExecutionResult object at 236ddf9efd0, execution_count=29 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 236ddeab990, raw_cell="#model_ft = train_model(model_ft, criterion, optim.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/c%3A/Users/ae423/OneDrive%20-%20University%20of%20Sussex/Desktop/AlphanElmasDissertation/Alphan_Elmas_Dissertation%20%281%29%20%281%29.ipynb#X42sZmlsZQ%3D%3D> result=None>,),kwargs {}:


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host